The dataset for this colab is available: [Training and validation set](https://drive.google.com/file/d/1Ji3oeXia4H4dd4bWpxkgTRfECS6Avn1d/view?usp=sharing) and [Test dataset](https://drive.google.com/file/d/1FWyWSnHNb28bit3mj2xrW5KpcvPKglGo/view?usp=sharing). 

The purpose of this study was to forecast the auction sale price of a specific piece of heavy equipment (a bulldozer) based on its usage, equipment type, and configuration. The information comes from auction result postings and covers use and equipment settings.
The RMSLE (root mean squared log error) between the actual and forecast auction prices is the competition's evaluation metric.

The data page contains links to sample submission files. Files submitted should be formatted as follows:

A newly created` header  `SalesID,SalePrice`;
has two columns:

`SalesID`: The validation set's SalesID in sorted order;
`SalePrice`: Your estimated sale price.

# [Root Mean Squared Log Error (RMSLE)](https://www.kaggle.com/code/carlolepelaars/understanding-the-metric-rmsle)

The formula for RMSLE is as follows:

$RMSLE=\sqrt{\frac{1}{n}\sum_{i=1}^{n}(log(p_i +1)-log(a_i +1))^2}$

where:

$n$  is the total number of observations in the (public/private) data set,

$p_i$  is the prediction of target, and

$a_i$  is the actual target for  $i$ .

$log(x)$  is the natural logarithm of  $x  ( log_e(x)$ .

#Prepare tools

##Random Forest Regression

Every decision tree has a significant variance, but when we mix all of them in parallel, the final variance is low since each decision tree is perfectly trained on that specific sample data, and so the outcome is dependent on numerous decision trees rather than one. In the case of a classification problem, the majority voting classifier is used to determine the final output. The final output of a regression problem is the mean of all the outputs. Aggregation is the name given to this section.

Random Forest is an ensemble technique that can handle both regression and classification tasks by combining many decision trees and a technique known as Bootstrap and Aggregation, or bagging. The core idea is to use numerous decision trees to determine the final output rather than depending on individual decision trees.

In [ ]:
# import exploratory data analysis and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# models from scikit-learn
from sklearn.ensemble import RandomForestRegressor

# model evaluations
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive

Mounted at /content/gdrive
/content/gdrive


#Inspect data

#Q1
Import the training and validation dataset `TrainAndValid.csv` with `low_memory=False`and priint first 5 rows of the dataset. What is the `auctionerrID` for `SalesID=1139246`? 



In [17]:
df = pd.read_csv('/content/gdrive/MyDrive/TrainAndValid.csv', low_memory=False)
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[df.SalesID == 1139246].auctioneerID

0    3.0
Name: auctioneerID, dtype: float64

#Q2
How many non-null values are there in the state column?



In [15]:
df['state'].notnull().sum()

412698

# Q3

Which of the following columns have missing values?

- MachineHoursCurrentMeter

- fiModelDesc

- fiBaseModel

- ProductSize

In [ ]:
df[['MachineHoursCurrentMeter','fiModelDesc','fiBaseModel','ProductSize']].isna().sum()

MachineHoursCurrentMeter    265194
fiModelDesc                      0
fiBaseModel                      0
ProductSize                 216605
dtype: int64

# Q4

How many columns are there in the TrainAndValid dataset?

In [ ]:
len(df.columns)

53

# Q5

Can you sort the dataframe in ascending order of saledate with inplace=True? Which of the following options represent the saledate of the salesID 1646770 ?

- 1989-01-17

- 1989-01-31

- 1992-03-25

- 2022-11-11

In [ ]:
df.sort_values(by='saledate', inplace=True)
df[df.SalesID == 1646770].saledate

205615    1/17/1989 0:00
Name: saledate, dtype: object

# Q6

Make a copy of the dataset. What is the year of making (YearMade) of the product with MachineID 1194089?

In [ ]:
df_new = df.copy()

df_new[df_new.MachineID == 1194089].YearMade

274835    1980
Name: YearMade, dtype: int64

# Q7

Separate the saledate in saleYear,saleMonth,saleDay,saleDayOfWeek and saleDayOfYear and saledate column. Which numeric columns from the following options have null values?

- saleDayOfWeek

- SalePrice

- auctioneerID

- MachineHoursCurrentMeter

In [ ]:
df_new.saledate = pd.to_datetime(df_new.saledate)

df_new['saleDay'] = df_new['saledate'].dt.day
df_new['saleMonth'] = df_new['saledate'].dt.month
df_new['saleYear'] = df_new['saledate'].dt.year
df_new['saleDayOfWeek'] = df_new['saledate'].dt.day_of_week
df_new['saleDayOfYear'] = df_new['saledate'].dt.day_of_year

In [ ]:
df_new[['saleDayOfWeek','SalePrice','auctioneerID','MachineHoursCurrentMeter']].isna().sum()

saleDayOfWeek                    0
SalePrice                        0
auctioneerID                 20136
MachineHoursCurrentMeter    265194
dtype: int64

# Q8 

How many values are missing in the column auctioneerID_is_missing?

In [ ]:
df_new.auctioneerID.isna().sum()

20136

# Q9

Convert the state and Usageband to numbers. What is the datatype?


[Hint: use pd.Categorical(df_tmp["feature_name"]).codes]

In [ ]:
df_new['state'] = pd.Categorical(df_new['state']).codes
df_new['UsageBand'] = pd.Categorical(df_new['UsageBand']).codes

df_new.state.dtype

dtype('int8')

# Q10

Add binary columns (with label+"_is_missing") to indicate whether sample had missing value.Turn all categories into numbers.All null values will be turned into "-1" so add +1 to all numbers. What is the value of Differential_Type_is_missing for SalesID=1646770?